In [1]:
import json
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import seaborn as sns

from textblob import TextBlob

%matplotlib inline

## 1. Populate Dataframes

In [2]:
# read from csv files
 
topicsDF = pd.read_csv(r'topics.csv',encoding='latin-1')

wf_twitterDF = pd.read_csv(r'twitter_wf_vader.csv',encoding='latin-1')
sap_twitterDF = pd.read_csv(r'twitter_sap_vader.csv',encoding='latin-1')

In [3]:
# Total number for each company
#wf_twitterDF= twitterDF[(twitterDF.source == 'WF') & (twitterDF.isRelated=='Y')]
print("WF Twits: "+ str(wf_twitterDF.size))

#sap_twitterDF = twitterDF[(twitterDF.source == 'SAP') & (twitterDF.isRelated=='Y')]
print("SAP Twits: "+ str(sap_twitterDF.size))

#ww_twitterDF = twitterDF[(twitterDF.source == 'WW') & (twitterDF.isRelated=='Y')]
#print("WeWork Twits: "+ str(ww_twitterDF.size))

WF Twits: 10074
SAP Twits: 12052


In [4]:
wf_twitterDF.head()

,Unnamed: 0,Unnamed: 0.1,id,text,URL,createdTimestamp,quoteCount,replyCount,retweetCount,favoriteCount,...,userFriendsCount,lang,isRelated,source,sourceDate,NP,isInReplyTo,user_id,vader_compound,vader_sentiment
0,0,0,5dab73ca11bb5a1f6c86dfbd,Wells Fargo names outsider as new CEO three ye...,https://t.co/WBtIAtX8Gf,2019-09-27 23:51:00,2,0,0,0,...,3918,en,Y,WF,2019-09-27,NaN,0.0,4.867858e+09,-0.6124,N
1,1,1,5dab73cb11bb5a1f6c86dfbe,Wells Fargo names outsider as new CEO three ye...,https://t.co/veyphufgzw,2019-09-27 23:47:53,0,0,0,0,...,1,en,Y,WF,2019-09-27,NaN,0.0,1.088562e+18,0.2500,E
2,2,2,5dab73cb11bb5a1f6c86dfbf,Charles Scharf as CEO Can Revive Wells Fargo. ...,https://t.co/7q3CanBgNk,2019-09-27 23:39:53,0,0,0,0,...,1023,en,Y,WF,2019-09-27,NaN,0.0,1.164775e+18,0.5267,P
3,3,3,5dab73cb11bb5a1f6c86dfc0,Wells Fargo Finally Appoints New CEO Who Is A ...,https://t.co/cghUhYeY8U,2019-09-27 23:39:38,0,0,0,0,...,367,en,Y,WF,2019-09-27,NaN,0.0,8.356552e+17,0.2500,E
4,4,4,5dab73cb11bb5a1f6c86dfc1,Wells Fargo gives new CEO a 40% pay boost to $...,https://t.co/1Dl0uMQxzV,2019-09-27 23:37:41,0,0,1,0,...,2481,en,Y,WF,2019-09-27,NaN,0.0,1.782049e+07,0.5106,P


## 2. Run through TextBlob Sentiment - WF

In [5]:
def get_sentiment_score (row):
    compound=np.nan
    if row['text'] is not None:
        textblob = TextBlob(row['text'])
        compound=textblob.sentiment.polarity
    return compound

def set_sentiment (row):
    #1. positive sentiment: compound score >= 0.5
    #2. neutral sentiment: (compound score > -0.5) and (compound score < 0.5)
    #3. negative sentiment: compound score <= -0.5
    sentiment= 'E' # Neutral
    upper_limit = 0.5
    lower_limit = -0.5
    if (row['tb_polarity'] >= upper_limit):
        sentiment = 'P' # Positive
    elif (row['tb_polarity'] <= lower_limit):
        sentiment = 'N' # Negative
        
    return sentiment
        

In [6]:
# https://stackoverflow.com/questions/26886653/pandas-create-new-column-based-on-values-from-other-columns-apply-a-function-o

wf_twitterDF['tb_polarity'] = wf_twitterDF.apply (lambda row: get_sentiment_score(row), axis=1)

In [7]:
wf_twitterDF['tb_sentiment'] = wf_twitterDF.apply (lambda row: set_sentiment(row), axis=1)

In [8]:
wf_twitterDF.head()

,Unnamed: 0,Unnamed: 0.1,id,text,URL,createdTimestamp,quoteCount,replyCount,retweetCount,favoriteCount,...,isRelated,source,sourceDate,NP,isInReplyTo,user_id,vader_compound,vader_sentiment,tb_polarity,tb_sentiment
0,0,0,5dab73ca11bb5a1f6c86dfbd,Wells Fargo names outsider as new CEO three ye...,https://t.co/WBtIAtX8Gf,2019-09-27 23:51:00,2,0,0,0,...,Y,WF,2019-09-27,NaN,0.0,4.867858e+09,-0.6124,N,-0.181818,E
1,1,1,5dab73cb11bb5a1f6c86dfbe,Wells Fargo names outsider as new CEO three ye...,https://t.co/veyphufgzw,2019-09-27 23:47:53,0,0,0,0,...,Y,WF,2019-09-27,NaN,0.0,1.088562e+18,0.2500,E,-0.181818,E
2,2,2,5dab73cb11bb5a1f6c86dfbf,Charles Scharf as CEO Can Revive Wells Fargo. ...,https://t.co/7q3CanBgNk,2019-09-27 23:39:53,0,0,0,0,...,Y,WF,2019-09-27,NaN,0.0,1.164775e+18,0.5267,P,0.000000,E
3,3,3,5dab73cb11bb5a1f6c86dfc0,Wells Fargo Finally Appoints New CEO Who Is A ...,https://t.co/cghUhYeY8U,2019-09-27 23:39:38,0,0,0,0,...,Y,WF,2019-09-27,NaN,0.0,8.356552e+17,0.2500,E,0.227841,E
4,4,4,5dab73cb11bb5a1f6c86dfc1,Wells Fargo gives new CEO a 40% pay boost to $...,https://t.co/1Dl0uMQxzV,2019-09-27 23:37:41,0,0,1,0,...,Y,WF,2019-09-27,NaN,0.0,1.782049e+07,0.5106,P,0.136364,E


In [9]:
# Save to csv for future analysis
wf_twitterDF.to_csv('twitter_wf_textblob.csv')

## 3. Run through TextBlob Sentiment - SAP

In [10]:
sap_twitterDF['tb_polarity'] = sap_twitterDF.apply (lambda row: get_sentiment_score(row), axis=1)

In [11]:
sap_twitterDF['tb_sentiment'] = sap_twitterDF.apply (lambda row: set_sentiment(row), axis=1)

In [12]:
sap_twitterDF.head()

,Unnamed: 0,Unnamed: 0.1,id,text,URL,createdTimestamp,quoteCount,replyCount,retweetCount,favoriteCount,...,isRelated,source,sourceDate,NP,isInReplyTo,user_id,vader_compound,vader_sentiment,tb_polarity,tb_sentiment
0,619,619,5dae77e811bb5a25b069085e,"Experience Management: ""Start with empathy for...",NaN,2019-10-09 18:10:03,0,0,0,0,...,Y,SAP,2019-10-09,NaN,0.0,2.593419e+08,0.0000,E,0.000000,E
1,621,621,5dae77e811bb5a25b0690860,"Experience Management: ""Start with empathy for...",https://t.co/t0MxMZ5DxI,2019-10-09 18:00:01,0,0,2,5,...,Y,SAP,2019-10-09,NaN,0.0,3.266873e+08,0.0000,E,0.000000,E
2,633,633,5dae77e911bb5a25b069086c,SAP CEO has made clear: the new economy is he...,https://t.co/tqPgEZix2U,2019-10-09 14:36:00,0,0,0,0,...,Y,SAP,2019-10-09,NaN,0.0,1.105129e+08,0.1027,E,0.078788,E
3,643,643,5dae77e911bb5a25b0690876,Thanks for hosting Bill Mcdermott CEO of SAP ...,NaN,2019-10-09 05:21:42,0,0,0,1,...,Y,SAP,2019-10-09,NaN,0.0,1.024890e+18,0.7184,P,0.200000,E
4,645,645,5dae784011bb5a25b0690878,"New article: ""SAP CEO Bill McDermott stepping ...",https://t.co/DoQvIY7sVy,2019-10-10 23:58:48,0,0,0,1,...,Y,SAP,2019-10-10,NaN,0.0,7.297547e+17,0.0000,E,-0.006397,E


In [13]:
# Save to csv for future analysis
sap_twitterDF.to_csv('twitter_sap_textblob.csv')